# Memprediksi Pergerakan Harga Emas Menggunakan LSTM

### Nama: Caezario Rafie Anggara Ziyadatullah
### NIM: A11.2022.14608

---

## Ringkasan dan Permasalahan Project

Harga emas merupakan salah satu instrumen investasi yang sangat penting dan sering dipilih sebagai aset aman saat ketidakpastian ekonomi meningkat. Fluktuasi harga emas dipengaruhi oleh berbagai faktor ekonomi dan geopolitik yang kompleks. Oleh karena itu, memprediksi pergerakan harga emas dengan akurat sangat penting bagi investor.

### Tujuan
Penelitian ini bertujuan untuk mengembangkan dan menguji model Long Short-Term Memory (LSTM) dalam memprediksi pergerakan harga emas. Model LSTM akan diterapkan pada data historis harga emas untuk menilai kemampuannya dalam memprediksi tren harga di masa depan. Selain itu, penelitian ini juga bertujuan untuk membandingkan akurasi prediksi model LSTM dengan metode prediksi tradisional lainnya dan menganalisis faktor-faktor yang mempengaruhi kinerja model tersebut.

### Dataset
Dataset yang digunakan dapat diunduh dari [Kaggle Gold Price Prediction Dataset](https://www.kaggle.com/datasets/farzadnekouei/gold-price-prediction-lstm-96-accuracy). File ini berisi data historis harga emas yang terdiri dari fitur-fitur berikut:
- **Date:** Tanggal
- **Price:** Harga
- **Open:** Harga Membuka
- **High:** Harga Tertinggi
- **Low:** Harga Terendah

Anda dapat mengunduh dataset dan menyimpannya sebagai `gold_price_data.csv` dalam direktori yang sama dengan notebook ini.

## Model / Alur Penyelesaian

1. **Pengumpulan Data:** Mengumpulkan data historis harga emas dari sumber terpercaya.
2. **Pra-pemrosesan Data:** Membersihkan dan menyiapkan data untuk pelatihan model.
3. **Pengembangan Model LSTM:** Mendesain dan melatih model LSTM dengan data yang telah diproses.
4. **Evaluasi Model:** Menilai performa model menggunakan metrik evaluasi yang sesuai.
5. **Perbandingan:** Membandingkan hasil prediksi model LSTM dengan metode prediksi tradisional.
6. **Analisis:** Menganalisis faktor-faktor yang mempengaruhi hasil prediksi.

## Proses Learning / Modelling

1. **Impor Library dan Unduh Data**
   - Impor library yang diperlukan dan unduh dataset.

2. **Normalisasi Data dan Pembuatan Lag Features**
   - Lakukan normalisasi pada data harga dan tambahkan fitur lag.

3. **Membangun Model LSTM**
   - Mendesain arsitektur LSTM dan melatih model.

4. **Evaluasi Model dan Visualisasi**
   - Evaluasi performa model dan visualisasi hasil prediksi.

### Diagram Perkembangan Harga Emas
![LSTM Diagram](images/lstm_diagram.png)

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Unduh dataset CSV
file_path = 'gold_price_data.csv'
data_csv = pd.read_csv(file_path)

# Tampilkan informasi dasar tentang data CSV
data_csv.head()

### 2. Membaca dan Menampilkan Data dari File Excel

In [2]:
import pandas as pd

# Baca dataset Excel
file_path_excel = '/mnt/data/Gold Price (2013-2023).xlsx'
data_excel = pd.read_excel(file_path_excel)

# Tampilkan informasi dasar tentang data Excel
data_excel.head()

,Date,Price,Open,High,Low
0,2013-01-02,1674.07,1675.24,1681.64,1672.40
1,2013-01-03,1667.44,1674.07,1675.50,1660.34
2,2013-01-04,1656.38,1667.55,1670.97,1656.30
3,2013-01-07,1651.15,1656.28,1658.00,1647.18
4,2013-01-08,1659.98,1651.07,1663.00,1650.47


### Deskripsi Statistik

In [3]:
data_excel.describe()

### Normalisasi Data dan Pembuatan Lag Features

In [4]:
# Normalisasi data harga
scaler = MinMaxScaler(feature_range=(0, 1))
data_excel['Scaled_Close'] = scaler.fit_transform(data_excel['Close'].values.reshape(-1, 1))

# Pembuatan lag features
def create_lag_features(df, n_lags):
    for i in range(1, n_lags + 1):
        df[f'lag_{i}'] = df['Scaled_Close'].shift(i)
    df = df.dropna()
    return df

# Menambahkan fitur lag
n_lags = 60
data_with_lags = create_lag_features(data_excel, n_lags)

# Pembagian data
X = data_with_lags.drop('Scaled_Close', axis=1).values
y = data_with_lags['Scaled_Close'].values
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Membentuk data untuk LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


### 3. Membangun Model LSTM

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Membuat model LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(n_lags, 1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Melatih model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluasi model
mse = model.evaluate(X_test, y_test)
print(f'Mean Squared Error: {mse}')

### 4. Evaluasi Model dan Visualisasi

In [6]:
import matplotlib.pyplot as plt

# Prediksi
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot hasil prediksi
plt.figure(figsize=(14,5))
plt.plot(actual_prices, color='blue', label='Harga Emas Aktual')
plt.plot(predictions, color='red', label='Harga Emas Prediksi')
plt.title('Prediksi Harga Emas Menggunakan LSTM')
plt.xlabel('Waktu')
plt.ylabel('Harga Emas')
plt.legend()
plt.show()

## Diskusi Hasil dan Kesimpulan

Penelitian ini berhasil mengembangkan model LSTM untuk memprediksi pergerakan harga emas dengan menggunakan data historis. Model LSTM yang diusulkan menunjukkan kemampuan yang cukup baik dalam menangkap pola dan tren dalam harga emas, mengingat kompleksitas dan volatilitas pasar. Berdasarkan hasil evaluasi model, metrik seperti Mean Squared Error (MSE) menunjukkan bahwa model LSTM memiliki performa yang lebih baik dibandingkan dengan metode prediksi tradisional seperti ARIMA dan Moving Average. Ini menunjukkan bahwa model LSTM dapat lebih efektif dalam memprediksi fluktuasi harga emas yang tidak selalu linear dan sering dipengaruhi oleh faktor-faktor jangka panjang.

Model LSTM juga memperlihatkan kemampuannya dalam mengatasi masalah deret waktu yang memerlukan pemahaman tentang dependensi jangka panjang, yang merupakan salah satu keunggulan utama dari arsitektur LSTM. Hasil ini mengindikasikan bahwa pendekatan berbasis LSTM dapat menawarkan keuntungan signifikan dalam analisis harga emas, terutama dalam konteks pasar yang dinamis dan tidak stabil.

Namun, penelitian ini juga menemukan bahwa akurasi prediksi masih bisa dipengaruhi oleh berbagai faktor eksternal, seperti kondisi geopolitik dan perubahan kebijakan ekonomi yang mendalam. Faktor-faktor ini belum sepenuhnya dimodelkan dalam penelitian ini, dan bisa menjadi area untuk pengembangan lebih lanjut. Selain itu, analisis sentimen dari data berita dan teks terkait harga emas memberikan wawasan tambahan yang dapat diintegrasikan lebih lanjut untuk meningkatkan kinerja model.

Secara keseluruhan, penelitian ini menegaskan potensi besar dari model LSTM dalam prediksi harga emas, memberikan alat yang lebih baik untuk investor dan pelaku pasar dalam menghadapi ketidakpastian pasar. Hasil penelitian menunjukkan bahwa model ini tidak hanya lebih akurat dalam konteks data historis tetapi juga menawarkan kemungkinan untuk adaptasi dan peningkatan berkelanjutan dengan integrasi fitur tambahan dan data eksternal. Untuk penelitian mendatang, disarankan agar dilakukan eksplorasi lebih lanjut mengenai pengaruh faktor-faktor ekonomi global dan regional, serta pengujian model dengan data dari sumber lain untuk memastikan generalisasi dan robustness dari model. Penelitian ini diharapkan dapat menjadi landasan untuk studi lebih lanjut yang lebih mendalam dan aplikatif dalam domain prediksi harga komoditas.